In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import pandas as pd

In [3]:
train = pd.read_csv("C:/dataset/dacon_SDSS/train.csv", index_col=0)
test = pd.read_csv("C:/dataset/dacon_SDSS/test.csv", index_col=0)
sample_submission = pd.read_csv("C:/dataset/dacon_SDSS/sample_submission.csv", index_col=0)

In [4]:
train2 = train.copy()
train2 = train2.drop(['fiberID'], axis=1)

In [5]:
test2 = test.copy()
test2 = test.drop(['fiberID'], axis=1)

In [6]:
# 레이블과 입력 테이블로 분리
y = train2.loc[:,'type']
X = train2.drop(['type'], axis=1)

In [7]:
# 문자열로된 이름에 번호를 붙이고(LabelEncoder) 그 번호를 원핫인코딩 방식으로 펼치기

from sklearn.preprocessing import LabelEncoder

encoder =  LabelEncoder()
y1 = encoder.fit_transform(y)
Y = pd.get_dummies(y1).values
Y

array([[0, 1, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 1, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0]], dtype=uint8)

In [8]:
# 스케일링
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

In [9]:
# 학습 데이터와 평가 데이터로 분리하기 
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(
                                    X_scaled, Y, test_size=0.2)

x_val, x_test, y_val, y_test = train_test_split( # train 60 val 20 test 20 으로 분할
    x_test, y_test, random_state=66, test_size=0.5)


In [10]:
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((159992, 20), (20000, 20), (159992, 19), (20000, 19))

In [11]:
# 모델의 설정
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
from keras.callbacks import EarlyStopping

model = Sequential()
model.add(Dense(256*3, input_dim=20, activation='relu'))
model.add(Dense(256*2, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(19, activation='softmax'))

model.summary()

Using TensorFlow backend.


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 768)               16128     
_________________________________________________________________
dense_2 (Dense)              (None, 512)               393728    
_________________________________________________________________
dense_3 (Dense)              (None, 256)               131328    
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 64)                16448     
_________________________________________________________________
dropout_2 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 32)               

In [12]:
# 모델 최적화 설정
early_stopping = EarlyStopping(monitor='val_loss', patience=30, mode='auto')

In [13]:
# 학습
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics = ['accuracy'])
model.fit(x_train, y_train, epochs=200, batch_size=256, 
         validation_data= (x_val, y_val), callbacks=[early_stopping] )


Train on 159992 samples, validate on 19999 samples
Epoch 1/200
159992/159992 [==============================] - 18s 113us/step - loss: 2.4206 - accuracy: 0.2266 - val_loss: 2.3139 - val_accuracy: 0.2496
Epoch 2/200
159992/159992 [==============================] - 17s 109us/step - loss: 2.3326 - accuracy: 0.2469 - val_loss: 2.3074 - val_accuracy: 0.2496
Epoch 3/200
159992/159992 [==============================] - 18s 111us/step - loss: 2.3193 - accuracy: 0.2484 - val_loss: 2.3068 - val_accuracy: 0.2496
Epoch 4/200
159992/159992 [==============================] - 18s 111us/step - loss: 2.3146 - accuracy: 0.2484 - val_loss: 2.3065 - val_accuracy: 0.2496
Epoch 5/200
159992/159992 [==============================] - 18s 111us/step - loss: 2.3119 - accuracy: 0.2484 - val_loss: 2.3063 - val_accuracy: 0.2496
Epoch 6/200
159992/159992 [==============================] - 18s 111us/step - loss: 2.3092 - accuracy: 0.2484 - val_loss: 2.3061 - val_accuracy: 0.2496
Epoch 7/200
159992/159992 [==========

In [14]:
# 평가하기
from sklearn.metrics import log_loss

loss, acc = model.evaluate(x_test, y_test, batch_size=1)
y_predict = model.predict(x_test)

print('loss : ', loss)
print('acc : ', acc)
print('y_predict(x_test) : \n', y_predict)
print('log loss : ', log_loss(y_test, y_predict))

20000/20000 [==============================] - 27s 1ms/step
loss :  2.3071784060657023
acc :  0.24709999561309814
y_predict(x_test) : 
 [[0.18716322 0.2459928  0.0730236  ... 0.06892324 0.00599397 0.01084951]
 [0.18716322 0.2459928  0.0730236  ... 0.06892324 0.00599397 0.01084951]
 [0.18716322 0.2459928  0.0730236  ... 0.06892324 0.00599397 0.01084951]
 ...
 [0.18716322 0.2459928  0.0730236  ... 0.06892324 0.00599397 0.01084951]
 [0.18716322 0.2459928  0.0730236  ... 0.06892324 0.00599397 0.01084951]
 [0.18716322 0.2459928  0.0730236  ... 0.06892324 0.00599397 0.01084951]]
log loss :  2.307178345477581


In [15]:
y_predict

array([[0.18716322, 0.2459928 , 0.0730236 , ..., 0.06892324, 0.00599397,
        0.01084951],
       [0.18716322, 0.2459928 , 0.0730236 , ..., 0.06892324, 0.00599397,
        0.01084951],
       [0.18716322, 0.2459928 , 0.0730236 , ..., 0.06892324, 0.00599397,
        0.01084951],
       ...,
       [0.18716322, 0.2459928 , 0.0730236 , ..., 0.06892324, 0.00599397,
        0.01084951],
       [0.18716322, 0.2459928 , 0.0730236 , ..., 0.06892324, 0.00599397,
        0.01084951],
       [0.18716322, 0.2459928 , 0.0730236 , ..., 0.06892324, 0.00599397,
        0.01084951]], dtype=float32)

In [16]:
test_pred = model.predict(test2)

In [17]:
test_pred

array([[2.3109405e-14, 1.0000000e+00, 1.8252771e-12, ..., 6.3836028e-12,
        3.1022601e-20, 3.8714034e-18],
       [1.8367616e-18, 1.0000000e+00, 7.2257363e-16, ..., 3.7643991e-15,
        1.1711166e-25, 5.2848701e-23],
       [3.7035057e-16, 1.0000000e+00, 5.8889600e-14, ..., 2.4592061e-13,
        1.3193081e-22, 2.8814171e-20],
       ...,
       [7.5508859e-20, 1.0000000e+00, 5.1163119e-17, ..., 3.0533200e-16,
        1.7132050e-27, 1.1967465e-24],
       [5.2097652e-24, 1.0000000e+00, 1.8012816e-20, ..., 1.6199960e-19,
        5.2597836e-33, 1.3711785e-29],
       [4.3199051e-16, 1.0000000e+00, 6.7112053e-14, ..., 2.7793626e-13,
        1.6203705e-22, 3.4652467e-20]], dtype=float32)

In [18]:
submission = pd.DataFrame(data=test_pred, columns=sample_submission.columns, index=sample_submission.index)
submission.to_csv('submission.csv', index=True)

In [19]:
submission.head()

,STAR_WHITE_DWARF,STAR_CATY_VAR,STAR_BROWN_DWARF,SERENDIPITY_RED,REDDEN_STD,STAR_BHB,GALAXY,SERENDIPITY_DISTANT,QSO,SKY,STAR_RED_DWARF,ROSAT_D,STAR_PN,SERENDIPITY_FIRST,STAR_CARBON,SPECTROPHOTO_STD,STAR_SUB_DWARF,SERENDIPITY_MANUAL,SERENDIPITY_BLUE
id,,,,,,,,,,,,,,,,,,,
199991,2.310941e-14,1.0,1.825277e-12,4.960944e-09,7.046752e-10,1.834345e-08,4.982130e-12,0.0,4.098463e-19,0.0,3.200067e-11,2.424227e-12,8.718924e-32,2.540179e-10,4.356609e-13,0.0,6.383603e-12,3.102260e-20,3.871403e-18
199992,1.836762e-18,1.0,7.225736e-16,2.758404e-11,1.502377e-12,1.659012e-10,3.360794e-15,0.0,2.647816e-24,0.0,3.038236e-14,1.078683e-15,0.000000e+00,7.110155e-13,1.437425e-16,0.0,3.764399e-15,1.171117e-25,5.284870e-23
199993,3.703506e-16,1.0,5.888960e-14,5.106970e-10,4.763423e-11,2.334689e-09,2.037960e-13,0.0,2.188656e-21,0.0,1.516511e-12,8.248392e-14,1.220683e-35,1.935992e-11,1.302519e-14,0.0,2.459206e-13,1.319308e-22,2.881417e-20
199994,6.719383e-20,1.0,4.649596e-17,4.475938e-12,1.741066e-13,3.192904e-11,2.604966e-16,0.0,4.018318e-26,0.0,2.655505e-15,7.229593e-17,0.000000e+00,9.082582e-14,8.653346e-18,0.0,2.787077e-16,1.467317e-27,1.041884e-24
199995,7.116314e-13,1.0,3.136102e-11,3.262895e-08,6.579507e-09,1.010993e-07,7.063703e-11,0.0,3.143614e-17,0.0,4.004418e-10,3.993593e-11,1.393139e-28,2.147160e-09,8.008236e-12,0.0,9.478547e-11,2.905736e-18,2.265402e-16


In [23]:
submission.to_csv('C:/Users/jieun/jieunnnnn/dacon_SDSS/submission2.csv', index=True)